In [45]:
import os
import json

objs = []
for root, dirs, files in os.walk("results"):
    for file in files:
        if file.endswith(".json"):
            with open(os.path.join(root, file)) as f:
                objs.append(json.load(f))

In [47]:
import pandas as pd

rows = []
for obj in objs:
    row = {}
    row['acc'] = obj['acc']
    for key in obj['args']:
        row[key] = obj['args'][key]
    rows.append(row)
df = pd.DataFrame(rows)

In [48]:
# Find all the unique values of the columns, if there is only one unique value, then we can drop the column
for col in df.columns:
    if isinstance(df[col][0], list) or len(df[col].unique()) == 1:
        df.drop(col, axis=1, inplace=True)
df

,acc,language,label,test_number,prompt_format,shot_number,model,device_map
0,44.0,ar,exp,100,MCoT-Two,0,XComposer2,cuda:3
1,85.0,en,exp,100,MCoT-Two,0,XComposer2,cuda:3
2,60.0,zh-CN,exp,100,MCoT-Two,0,XComposer2,cuda:3
3,70.0,de,exp,100,MCoT-Two,0,XComposer2,cuda:3
4,57.0,bg,exp,100,MCoT-Two,0,XComposer2,cuda:3
...,...,...,...,...,...,...,...,...
82,85.0,th,gpt4_0shot,20,MCoT-Two,0,GPT4V,cuda:3
83,85.0,sw,gpt4_0shot,20,MCoT-Two,0,GPT4V,cuda:3
84,80.0,ru,gpt4_0shot,20,MCoT-Two,0,GPT4V,cuda:3
85,80.0,fr,gpt4_0shot,20,MCoT-Two,0,GPT4V,cuda:3


In [52]:
display_tab = {}
for idx, row in df.iterrows():
    model = row['model'].split('/')[-1]
    shot_number = row['shot_number']
    prompt_format = row['prompt_format']
    language = row['language']
    name = f"{model}-{prompt_format}-{shot_number}Shot"
    if name not in display_tab:
        display_tab[name] = {}
    display_tab[name][language] = row['acc']

display_df = pd.DataFrame(display_tab)
display_df.transpose().sort_index()
# replace nan with '-'
display_df = display_df.fillna('-')
# .2f for 2 decimal places
display_df = display_df.map(lambda x: f"{x:.0f}" if x != '-' else x)
# to markdown
markdown = display_df.transpose().to_markdown()
display_df.transpose().sort_index()

,ar,en,zh-CN,de,bg,tr,hi,es,el,th,sw,ru,fr,vi,ur
GPT4V-MCoT-Two-0Shot,75,100,90,80,85,85,80,80,75,85,85,80,80,-,-
GeminiProVision-MCoT-One-3Shot,-,47,70,-,-,-,-,-,-,-,-,-,-,-,-
GeminiProVision-MCoT-Two-0Shot,56,78,74,-,-,-,-,-,-,-,-,-,-,-,-
Qwen-VL-EnCoT-Two-0Shot,38,62,50,56,52,47,43,56,37,42,51,44,60,-,-
Qwen-VL-EnCoT-Two-3Shot,33,44,38,47,40,-,-,-,42,-,-,-,-,-,-
Qwen-VL-MCoT-Two-0Shot,30,56,49,45,42,38,39,41,36,35,39,38,40,45,37
Qwen-VL-MCoT-Two-3Shot,43,53,43,45,30,33,31,48,38,33,37,44,43,34,35
XComposer2-MCoT-Two-0Shot,44,85,60,70,57,-,-,-,-,-,-,-,-,-,-
mblip-bloomz-7b-MCoT-Two-0Shot,41,45,34,40,34,34,34,41,34,34,37,34,40,43,33


In [50]:
notes = """
所有数据集都只跑了 100 条，全部跑完比较慢。其中 GPT4V 每个数据集只跑了 20 条。

模型：
- XComposer2: 也就是 InternVL: internlm/internlm-xcomposer2-vl-7b
- GeminiProVision：GeminiProVision 的 API
- Qwen-VL：Qwen VL-8B 的 Base 版本：Qwen/Qwen-VL
- mblip-bloomz-7b：就这个是 7b 左右的 Gregor/mblip-bloomz-7b
- GPT4V：GPT4V 的 API
mPLUG-Owl2 太难配置了，就还没跑

Prompt 格式：
- MCoT：使用和数据对应的语言作为 CoT 的输入
- EnCoT：使用英文 CoT 提示词
- *-One/Two：是否使用两阶段提示。如果是 One 则直接从模型的输出里用一些启发式的方法抽取答案。如果是 Two 就再给模型一个 Prompt：（Therefore the answer is）然后再让模型输出答案。

Few-shot：Few-shot 用固定的随机数种子随机选几个样本。所有的输入都是一样的 Few-shot 样本。
"""

print(notes + markdown)


所有数据集都只跑了 100 条，全部跑完比较慢。其中 GPT4V 每个数据集只跑了 20 条。

模型：
- XComposer2: 也就是 InternVL: internlm/internlm-xcomposer2-vl-7b
- GeminiProVision：GeminiProVision 的 API
- Qwen-VL：Qwen VL-8B 的 Base 版本：Qwen/Qwen-VL
- mblip-bloomz-7b：就这个是 7b 左右的 Gregor/mblip-bloomz-7b
- GPT4V：GPT4V 的 API
mPLUG-Owl2 太难配置了，就还没跑

Prompt 格式：
- MCoT：使用和数据对应的语言作为 CoT 的输入
- EnCoT：使用英文 CoT 提示词
- *-One/Two：是否使用两阶段提示。如果是 One 则直接从模型的输出里用一些启发式的方法抽取答案。如果是 Two 就再给模型一个 Prompt：（Therefore the answer is）然后再让模型输出答案。

Few-shot：Few-shot 用固定的随机数种子随机选几个样本。所有的输入都是一样的 Few-shot 样本。
|                                | ar   |   en |   zh-CN | de   | bg   | tr   | hi   | es   | el   | th   | sw   | ru   | fr   | vi   | ur   |
|:-------------------------------|:-----|-----:|--------:|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
| XComposer2-MCoT-Two-0Shot      | 44   |   85 |      60 | 70   | 57   | -    | -    | -    | -    | -    | -    | -    | -    | -    | -    |
| GeminiProVision-MCoT-Two-0Shot | 56   